In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

In [4]:
# removing location column
def remove_location(df):
    return df.drop(columns=['City'], inplace=True)

In [5]:
# converting date column to datetime
def date_to_datetime(df):
    df['Current_date'] = pd.to_datetime(df['Current_date'])
    return df

## Colombo

In [6]:
colombo = pd.read_csv('Preprocessed-Data/colombo.csv')
colombo

HCHO_emission     City Current_date
0          0.000263  Colombo   2019-01-02
1          0.000099  Colombo   2019-01-03
2          0.000210  Colombo   2019-01-04
3          0.000179  Colombo   2019-01-05
4          0.000108  Colombo   2019-01-06
...             ...      ...          ...
1316       0.000182  Colombo   2023-12-24
1317       0.000343  Colombo   2023-12-25
1318       0.000118  Colombo   2023-12-27
1319       0.000140  Colombo   2023-12-30
1320       0.000080  Colombo   2023-12-31

[1321 rows x 3 columns]

In [7]:
remove_location(colombo)
date_to_datetime(colombo)
colombo

HCHO_emission Current_date
0          0.000263   2019-01-02
1          0.000099   2019-01-03
2          0.000210   2019-01-04
3          0.000179   2019-01-05
4          0.000108   2019-01-06
...             ...          ...
1316       0.000182   2023-12-24
1317       0.000343   2023-12-25
1318       0.000118   2023-12-27
1319       0.000140   2023-12-30
1320       0.000080   2023-12-31

[1321 rows x 2 columns]

In [8]:
split_colombo = int(len(colombo) * 0.8) # 80% training split
train_split_colombo, test_split_colombo = colombo.iloc[:split_colombo], colombo.iloc[split_colombo:] # splitting to training and testing

colombo_auto_arima = auto_arima(train_split_colombo['HCHO_emission'], seasonal=True, trace=True, m=12) # best parameters are obtained using autoarima
# p- autoregressive order
# d- differencing order/ integration order
# q- moving average order
p,d,q = colombo_auto_arima.order
colombo_arima = ARIMA(train_split_colombo['HCHO_emission'], order=(p,d,q)) # creating an ARIMA model
colombo_arima_fit = colombo_arima.fit() # fitting the model

colombo_testing_predictions = colombo_arima_fit.forecast(steps=len(test_split_colombo)) # taking predictions on testing data
colombo_mse = mean_squared_error(test_split_colombo['HCHO_emission'], colombo_testing_predictions) # calculating mean squared error
print("Mean Squared Error for Colombo: ", colombo_mse)
print(colombo_testing_predictions)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=-17074.601, Time=4.83 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=-16553.784, Time=0.21 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=-16824.431, Time=0.48 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=-17045.221, Time=2.23 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=-16555.788, Time=0.15 sec
 ARIMA(2,1,2)(0,0,1)[12] intercept   : AIC=-17076.316, Time=0.82 sec
 ARIMA(2,1,2)(0,0,0)[12] intercept   : AIC=-17080.376, Time=0.41 sec
 ARIMA(2,1,2)(1,0,0)[12] intercept   : AIC=-17078.511, Time=0.87 sec
 ARIMA(1,1,2)(0,0,0)[12] intercept   : AIC=-17080.645, Time=0.36 sec
 ARIMA(1,1,2)(1,0,0)[12] intercept   : AIC=-17078.795, Time=0.66 sec
 ARIMA(1,1,2)(0,0,1)[12] intercept   : AIC=-17076.813, Time=0.84 sec
 ARIMA(1,1,2)(1,0,1)[12] intercept   : AIC=-17075.105, Time=3.79 sec
 ARIMA(0,1,2)(0,0,0)[12] intercept   : AIC=-17087.056, Time=0.33 sec
 ARIMA(0,1,2)(1,0,0)[12] intercept   : AIC=-17085.178, Time=

c:\Users\MSI\Downloads\Datasets\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [9]:
# forecasting for further 30 days to display on dashboard
forecasting_days =30
colombo_forecast_range = pd.date_range(start=colombo['Current_date'].max(), periods=forecasting_days+1, freq='D')[1:] # gives the dates of prediction
colombo_forecast_values = colombo_arima_fit.forecast(steps=forecasting_days) # gives the predicted HCHO emission values

#getting the predictions onto a data frame
colombo_forecast_df = pd.DataFrame({'Prediction_date': colombo_forecast_range, 'HCHO_emission': colombo_forecast_values, 'Location': 'Colombo'})
colombo_forecast_df.set_index('Prediction_date', inplace=True)
colombo_forecast_df

HCHO_emission Location
Prediction_date                        
2024-01-01            0.000128  Colombo
2024-01-02            0.000128  Colombo
2024-01-03            0.000135  Colombo
2024-01-04            0.000131  Colombo
2024-01-05            0.000134  Colombo
2024-01-06            0.000132  Colombo
2024-01-07            0.000133  Colombo
2024-01-08            0.000133  Colombo
2024-01-09            0.000133  Colombo
2024-01-10            0.000133  Colombo
2024-01-11            0.000133  Colombo
2024-01-12            0.000133  Colombo
2024-01-13            0.000133  Colombo
2024-01-14            0.000133  Colombo
2024-01-15            0.000133  Colombo
2024-01-16            0.000133  Colombo
2024-01-17            0.000133  Colombo
2024-01-18            0.000133  Colombo
2024-01-19            0.000133  Colombo
2024-01-20            0.000133  Colombo
2024-01-21            0.000133  Colombo
2024-01-22            0.000133  Colombo
2024-01-23            0.000133  Colombo
2024-01-24            0.000133  Colombo
2024-01-25            0.000133  Colombo
2024-01-26            0.000133  Colombo
2024-01-27            0.000133  Colombo
2024-01-28            0.000133  Colombo
2024-01-29            0.000133  Colombo
2024-01-30            0.000133  Colombo

In [10]:
folder = 'Forecasted-Predictions'

if not os.path.exists(folder):
    os.makedirs(folder)

colombo_forecast_df.to_csv(f'{folder}/colombo_forecast.csv') # save the forecasted data to a csv file

## Nuwara Eliya

In [11]:
nuwara_eliya = pd.read_csv('Preprocessed-Data/nuwaraeliya.csv')
nuwara_eliya

HCHO_emission          City Current_date
0         0.000191  Nuwara Eliya   2019-01-03
1         0.000051  Nuwara Eliya   2019-01-04
2         0.000065  Nuwara Eliya   2019-01-05
3         0.000090  Nuwara Eliya   2019-01-06
4         0.000041  Nuwara Eliya   2019-01-07
..             ...           ...          ...
724       0.000036  Nuwara Eliya   2023-11-23
725       0.000103  Nuwara Eliya   2023-12-10
726       0.000108  Nuwara Eliya   2023-12-13
727       0.000006  Nuwara Eliya   2023-12-17
728       0.000237  Nuwara Eliya   2023-12-23

[729 rows x 3 columns]

In [12]:
remove_location(nuwara_eliya)
date_to_datetime(nuwara_eliya)
nuwara_eliya

HCHO_emission Current_date
0         0.000191   2019-01-03
1         0.000051   2019-01-04
2         0.000065   2019-01-05
3         0.000090   2019-01-06
4         0.000041   2019-01-07
..             ...          ...
724       0.000036   2023-11-23
725       0.000103   2023-12-10
726       0.000108   2023-12-13
727       0.000006   2023-12-17
728       0.000237   2023-12-23

[729 rows x 2 columns]

In [13]:
split_nuwara_eliya = int(len(nuwara_eliya) * 0.8) # 80% training split
train_split_nuwara_eliya, test_split_nuwara_eliya = nuwara_eliya.iloc[:split_nuwara_eliya], nuwara_eliya.iloc[split_nuwara_eliya:] # splitting to training and testing

nuwara_eliya_auto_arima = auto_arima(train_split_nuwara_eliya['HCHO_emission'], seasonal=True, trace=True, m=12) # best parameters are obtained using autoarima
p,d,q = nuwara_eliya_auto_arima.order
nuwara_eliya_arima = ARIMA(train_split_nuwara_eliya['HCHO_emission'], order=(p,d,q)) # creating an ARIMA model
nuwara_eliya_arima_fit = nuwara_eliya_arima.fit() # fitting the model

nuwara_eliya_testing_predictions = nuwara_eliya_arima_fit.forecast(steps=len(test_split_nuwara_eliya)) # taking predictions on testing data
nuwara_eliya_mse = mean_squared_error(test_split_nuwara_eliya['HCHO_emission'], nuwara_eliya_testing_predictions) # calculating mean squared error
print("Mean Squared Error for Nuwara Eliya: ", nuwara_eliya_mse)
print(nuwara_eliya_testing_predictions)

Performing stepwise search to minimize aic


 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=inf, Time=3.89 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=-9659.604, Time=0.06 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=-9407.146, Time=0.70 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=-9658.076, Time=0.34 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=-8868.885, Time=0.03 sec
 ARIMA(0,0,0)(1,0,0)[12] intercept   : AIC=-9404.070, Time=0.44 sec
 ARIMA(0,0,0)(0,0,1)[12] intercept   : AIC=-9657.038, Time=0.24 sec
 ARIMA(0,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=1.55 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=-9660.724, Time=0.08 sec
 ARIMA(1,0,0)(0,0,1)[12] intercept   : AIC=-9657.841, Time=0.35 sec
 ARIMA(1,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=1.39 sec
 ARIMA(2,0,0)(0,0,0)[12] intercept   : AIC=-9660.514, Time=0.12 sec
 ARIMA(1,0,1)(0,0,0)[12] intercept   : AIC=-9658.877, Time=0.19 sec
 ARIMA(0,0,1)(0,0,0)[12] intercept   : AIC=-9660.850, Time=0.13 sec
 ARIMA(0,0,1)(1,0,0)[12] intercept   : AIC=-9407.917, Time=0.74 se

In [14]:
# forecasting for further 30 days to display on dashboard
forecasting_days =30
nuwara_eliya_forecast_range = pd.date_range(start=nuwara_eliya['Current_date'].max(), periods=forecasting_days+1, freq='D')[1:] # gives the dates of prediction
nuwara_eliya_forecast_values = nuwara_eliya_arima_fit.forecast(steps=forecasting_days) # gives the predicted HCHO emission values

#getting the predictions onto a data frame
nuwara_eliya_forecast_df = pd.DataFrame({'Prediction_date': nuwara_eliya_forecast_range, 'HCHO_emission': nuwara_eliya_forecast_values, 'Location': 'nuwara_eliya'})
nuwara_eliya_forecast_df.set_index('Prediction_date', inplace=True)
nuwara_eliya_forecast_df

HCHO_emission      Location
Prediction_date                             
2023-12-24            0.000096  nuwara_eliya
2023-12-25            0.000104  nuwara_eliya
2023-12-26            0.000104  nuwara_eliya
2023-12-27            0.000104  nuwara_eliya
2023-12-28            0.000104  nuwara_eliya
2023-12-29            0.000104  nuwara_eliya
2023-12-30            0.000104  nuwara_eliya
2023-12-31            0.000104  nuwara_eliya
2024-01-01            0.000104  nuwara_eliya
2024-01-02            0.000104  nuwara_eliya
2024-01-03            0.000104  nuwara_eliya
2024-01-04            0.000104  nuwara_eliya
2024-01-05            0.000104  nuwara_eliya
2024-01-06            0.000104  nuwara_eliya
2024-01-07            0.000104  nuwara_eliya
2024-01-08            0.000104  nuwara_eliya
2024-01-09            0.000104  nuwara_eliya
2024-01-10            0.000104  nuwara_eliya
2024-01-11            0.000104  nuwara_eliya
2024-01-12            0.000104  nuwara_eliya
2024-01-13            0.000104  nuwara_eliya
2024-01-14            0.000104  nuwara_eliya
2024-01-15            0.000104  nuwara_eliya
2024-01-16            0.000104  nuwara_eliya
2024-01-17            0.000104  nuwara_eliya
2024-01-18            0.000104  nuwara_eliya
2024-01-19            0.000104  nuwara_eliya
2024-01-20            0.000104  nuwara_eliya
2024-01-21            0.000104  nuwara_eliya
2024-01-22            0.000104  nuwara_eliya

In [15]:
folder = 'Forecasted-Predictions'

if not os.path.exists(folder):
    os.makedirs(folder)

nuwara_eliya_forecast_df.to_csv(f'{folder}/nuwara_eliya_forecast.csv') # save the forecasted data to a csv file

## Matara

In [16]:
matara = pd.read_csv('Preprocessed-Data/matara.csv')
matara

HCHO_emission    City Current_date
0         0.000006  Matara   2019-01-02
1         0.000236  Matara   2019-01-03
2         0.000064  Matara   2019-01-04
3         0.000053  Matara   2019-01-05
4         0.000132  Matara   2019-01-06
..             ...     ...          ...
948       0.000063  Matara   2023-12-24
949       0.000072  Matara   2023-12-25
950       0.000059  Matara   2023-12-26
951       0.000228  Matara   2023-12-27
952       0.000116  Matara   2023-12-30

[953 rows x 3 columns]

In [17]:
remove_location(matara)
date_to_datetime(matara)
matara

HCHO_emission Current_date
0         0.000006   2019-01-02
1         0.000236   2019-01-03
2         0.000064   2019-01-04
3         0.000053   2019-01-05
4         0.000132   2019-01-06
..             ...          ...
948       0.000063   2023-12-24
949       0.000072   2023-12-25
950       0.000059   2023-12-26
951       0.000228   2023-12-27
952       0.000116   2023-12-30

[953 rows x 2 columns]

In [18]:
split_matara = int(len(matara) * 0.8) # 80% training split
train_split_matara, test_split_matara = matara.iloc[:split_matara], matara.iloc[split_matara:] # splitting to training and testing

matara_auto_arima = auto_arima(train_split_matara['HCHO_emission'], seasonal=True, trace=True, m=12) # best parameters are obtained using autoarima
p,d,q = matara_auto_arima.order
matara_arima = ARIMA(train_split_matara['HCHO_emission'], order=(p,d,q)) # creating an ARIMA model
matara_arima_fit = matara_arima.fit() # fitting the model

matara_testing_predictions = matara_arima_fit.forecast(steps=len(test_split_matara)) # taking predictions on testing data
matara_mse = mean_squared_error(test_split_matara['HCHO_emission'], matara_testing_predictions) # calculating mean squared error
print("Mean Squared Error for Matara: ", matara_mse)
print(matara_testing_predictions)

Performing stepwise search to minimize aic


 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=inf, Time=3.84 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=-12685.246, Time=0.06 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=-12362.101, Time=0.97 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=-12684.376, Time=0.38 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=-11645.489, Time=0.03 sec
 ARIMA(0,0,0)(1,0,0)[12] intercept   : AIC=-12361.519, Time=0.65 sec
 ARIMA(0,0,0)(0,0,1)[12] intercept   : AIC=-12684.483, Time=0.72 sec
 ARIMA(0,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=1.30 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=-12686.443, Time=0.08 sec
 ARIMA(1,0,0)(0,0,1)[12] intercept   : AIC=-12685.636, Time=0.95 sec
 ARIMA(1,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=1.49 sec
 ARIMA(2,0,0)(0,0,0)[12] intercept   : AIC=-12686.575, Time=0.18 sec
 ARIMA(2,0,0)(1,0,0)[12] intercept   : AIC=-12364.564, Time=1.16 sec
 ARIMA(2,0,0)(0,0,1)[12] intercept   : AIC=-12685.999, Time=0.54 sec
 ARIMA(2,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=2.

c:\Users\MSI\Downloads\Datasets\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [19]:
# forecasting for further 30 days to display on dashboard
forecasting_days =30
matara_forecast_range = pd.date_range(start=matara['Current_date'].max(), periods=forecasting_days+1, freq='D')[1:] # gives the dates of prediction
matara_forecast_values = matara_arima_fit.forecast(steps=forecasting_days) # gives the predicted HCHO emission values

#getting the predictions onto a data frame
matara_forecast_df = pd.DataFrame({'Prediction_date': matara_forecast_range, 'HCHO_emission': matara_forecast_values, 'Location': 'matara'})
matara_forecast_df.set_index('Prediction_date', inplace=True)
matara_forecast_df

HCHO_emission Location
Prediction_date                        
2023-12-31            0.000101   matara
2024-01-01            0.000099   matara
2024-01-02            0.000100   matara
2024-01-03            0.000100   matara
2024-01-04            0.000100   matara
2024-01-05            0.000100   matara
2024-01-06            0.000100   matara
2024-01-07            0.000100   matara
2024-01-08            0.000100   matara
2024-01-09            0.000100   matara
2024-01-10            0.000100   matara
2024-01-11            0.000100   matara
2024-01-12            0.000100   matara
2024-01-13            0.000100   matara
2024-01-14            0.000100   matara
2024-01-15            0.000100   matara
2024-01-16            0.000100   matara
2024-01-17            0.000100   matara
2024-01-18            0.000100   matara
2024-01-19            0.000100   matara
2024-01-20            0.000100   matara
2024-01-21            0.000100   matara
2024-01-22            0.000100   matara
2024-01-23            0.000100   matara
2024-01-24            0.000100   matara
2024-01-25            0.000100   matara
2024-01-26            0.000100   matara
2024-01-27            0.000100   matara
2024-01-28            0.000100   matara
2024-01-29            0.000100   matara

In [20]:
folder = 'Forecasted-Predictions'

if not os.path.exists(folder):
    os.makedirs(folder)

matara_forecast_df.to_csv(f'{folder}/matara_forecast.csv') # save the forecasted data to a csv file

## Kandy

In [21]:
kandy = pd.read_csv('Preprocessed-Data/kandy.csv')
kandy

HCHO_emission   City Current_date
0          0.000092  Kandy   2019-01-02
1          0.000191  Kandy   2019-01-04
2          0.000122  Kandy   2019-01-05
3          0.000121  Kandy   2019-01-06
4          0.000163  Kandy   2019-01-07
...             ...    ...          ...
1016       0.000121  Kandy   2023-12-17
1017       0.000114  Kandy   2023-12-21
1018       0.000121  Kandy   2023-12-22
1019       0.000116  Kandy   2023-12-23
1020       0.000056  Kandy   2023-12-30

[1021 rows x 3 columns]

In [22]:
remove_location(kandy)
date_to_datetime(kandy)
kandy

HCHO_emission Current_date
0          0.000092   2019-01-02
1          0.000191   2019-01-04
2          0.000122   2019-01-05
3          0.000121   2019-01-06
4          0.000163   2019-01-07
...             ...          ...
1016       0.000121   2023-12-17
1017       0.000114   2023-12-21
1018       0.000121   2023-12-22
1019       0.000116   2023-12-23
1020       0.000056   2023-12-30

[1021 rows x 2 columns]

In [23]:
split_kandy = int(len(kandy) * 0.8) # 80% training split
train_split_kandy, test_split_kandy = kandy.iloc[:split_kandy], kandy.iloc[split_kandy:] # splitting to training and testing
kandy_auto_arima = auto_arima(train_split_kandy['HCHO_emission'], seasonal=True, trace=True, m=12) # best parameters are obtained using autoarima

p,d,q = kandy_auto_arima.order

kandy_arima = ARIMA(train_split_kandy['HCHO_emission'], order=(p,d,q)) # creating an ARIMA model
kandy_arima_fit = kandy_arima.fit() # fitting the model

kandy_testing_predictions = kandy_arima_fit.forecast(steps=len(test_split_kandy)) # taking predictions on testing data

kandy_mse = mean_squared_error(test_split_kandy['HCHO_emission'], kandy_testing_predictions) # calculating mean squared error
print("Mean Squared Error for kandy: ", kandy_mse)

print(kandy_testing_predictions)

Performing stepwise search to minimize aic


 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=-13375.016, Time=0.92 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=-12995.027, Time=0.11 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=-13160.477, Time=0.96 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=-13311.114, Time=2.24 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=-12996.749, Time=0.05 sec
 ARIMA(2,1,2)(0,0,1)[12] intercept   : AIC=-13377.022, Time=0.75 sec
 ARIMA(2,1,2)(0,0,0)[12] intercept   : AIC=-13378.294, Time=0.42 sec
 ARIMA(2,1,2)(1,0,0)[12] intercept   : AIC=-13377.031, Time=0.87 sec
 ARIMA(1,1,2)(0,0,0)[12] intercept   : AIC=-13367.598, Time=0.29 sec
 ARIMA(2,1,1)(0,0,0)[12] intercept   : AIC=-13312.340, Time=0.27 sec
 ARIMA(3,1,2)(0,0,0)[12] intercept   : AIC=-13383.048, Time=0.37 sec
 ARIMA(3,1,2)(1,0,0)[12] intercept   : AIC=-13379.605, Time=0.86 sec
 ARIMA(3,1,2)(0,0,1)[12] intercept   : AIC=-13380.470, Time=0.85 sec
 ARIMA(3,1,2)(1,0,1)[12] intercept   : AIC=-13378.608, Time=0.95 sec
 ARIMA(3,1,1)(0,0,0)[12] intercept

c:\Users\MSI\Downloads\Datasets\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [24]:
# forecasting for further 30 days to display on dashboard
forecasting_days =30
kandy_forecast_range = pd.date_range(start=kandy['Current_date'].max(), periods=forecasting_days+1, freq='D')[1:] # gives the dates of prediction
kandy_forecast_values = kandy_arima_fit.forecast(steps=forecasting_days) # gives the predicted HCHO emission values

#getting the predictions onto a data frame
kandy_forecast_df = pd.DataFrame({'Prediction_date': kandy_forecast_range, 'HCHO_emission': kandy_forecast_values, 'Location': 'kandy'})
kandy_forecast_df.set_index('Prediction_date', inplace=True)
kandy_forecast_df

HCHO_emission Location
Prediction_date                        
2023-12-31            0.000089    kandy
2024-01-01            0.000103    kandy
2024-01-02            0.000114    kandy
2024-01-03            0.000107    kandy
2024-01-04            0.000103    kandy
2024-01-05            0.000108    kandy
2024-01-06            0.000109    kandy
2024-01-07            0.000106    kandy
2024-01-08            0.000106    kandy
2024-01-09            0.000108    kandy
2024-01-10            0.000107    kandy
2024-01-11            0.000106    kandy
2024-01-12            0.000107    kandy
2024-01-13            0.000107    kandy
2024-01-14            0.000107    kandy
2024-01-15            0.000107    kandy
2024-01-16            0.000107    kandy
2024-01-17            0.000107    kandy
2024-01-18            0.000107    kandy
2024-01-19            0.000107    kandy
2024-01-20            0.000107    kandy
2024-01-21            0.000107    kandy
2024-01-22            0.000107    kandy
2024-01-23            0.000107    kandy
2024-01-24            0.000107    kandy
2024-01-25            0.000107    kandy
2024-01-26            0.000107    kandy
2024-01-27            0.000107    kandy
2024-01-28            0.000107    kandy
2024-01-29            0.000107    kandy

In [25]:
folder = 'Forecasted-Predictions'

if not os.path.exists(folder):
    os.makedirs(folder)

kandy_forecast_df.to_csv(f'{folder}/kandy_forecast.csv') # save the forecasted data to a csv file

## Monaragala

In [26]:
monaragala = pd.read_csv('Preprocessed-Data/monaragala.csv')
monaragala

HCHO_emission        City Current_date
0          0.000019  Monaragala   2019-01-02
1          0.000028  Monaragala   2019-01-03
2          0.000037  Monaragala   2019-01-04
3          0.000136  Monaragala   2019-01-05
4          0.000146  Monaragala   2019-01-06
...             ...         ...          ...
1109       0.000052  Monaragala   2023-12-18
1110       0.000136  Monaragala   2023-12-20
1111       0.000136  Monaragala   2023-12-21
1112       0.000149  Monaragala   2023-12-23
1113       0.000031  Monaragala   2023-12-25

[1114 rows x 3 columns]

In [27]:
remove_location(monaragala)
date_to_datetime(monaragala)
monaragala

HCHO_emission Current_date
0          0.000019   2019-01-02
1          0.000028   2019-01-03
2          0.000037   2019-01-04
3          0.000136   2019-01-05
4          0.000146   2019-01-06
...             ...          ...
1109       0.000052   2023-12-18
1110       0.000136   2023-12-20
1111       0.000136   2023-12-21
1112       0.000149   2023-12-23
1113       0.000031   2023-12-25

[1114 rows x 2 columns]

In [28]:
split_monaragala = int(len(monaragala) * 0.8) # 80% training split
train_split_monaragala, test_split_monaragala = monaragala.iloc[:split_monaragala], monaragala.iloc[split_monaragala:] # splitting to training and testing

monaragala_auto_arima = auto_arima(train_split_monaragala['HCHO_emission'], seasonal=True, trace=True, m=12) # best parameters are obtained using autoarima
p,d,q = monaragala_auto_arima.order
monaragala_arima = ARIMA(train_split_monaragala['HCHO_emission'], order=(p,d,q)) # creating an ARIMA model
monaragala_arima_fit = monaragala_arima.fit() # fitting the model

monaragala_testing_predictions = monaragala_arima_fit.forecast(steps=len(test_split_monaragala)) # taking predictions on testing data
monaragala_mse = mean_squared_error(test_split_monaragala['HCHO_emission'], monaragala_testing_predictions) # calculating mean squared error
print("Mean Squared Error for Monaragala: ", monaragala_mse)
print(monaragala_testing_predictions)

Performing stepwise search to minimize aic


 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=inf, Time=3.62 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=-14525.151, Time=0.06 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=-14181.000, Time=1.29 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=-14547.322, Time=1.11 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=-13139.770, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[12] intercept   : AIC=-14538.085, Time=0.43 sec
 ARIMA(0,0,1)(1,0,1)[12] intercept   : AIC=inf, Time=2.30 sec
 ARIMA(0,0,1)(0,0,2)[12] intercept   : AIC=-14554.616, Time=3.43 sec
 ARIMA(0,0,1)(1,0,2)[12] intercept   : AIC=-12622.292, Time=6.90 sec
 ARIMA(0,0,0)(0,0,2)[12] intercept   : AIC=-14544.250, Time=2.51 sec
 ARIMA(1,0,1)(0,0,2)[12] intercept   : AIC=-14571.411, Time=3.16 sec
 ARIMA(1,0,1)(0,0,1)[12] intercept   : AIC=-14569.594, Time=1.54 sec
 ARIMA(1,0,1)(1,0,2)[12] intercept   : AIC=-14516.022, Time=9.90 sec
 ARIMA(1,0,1)(1,0,1)[12] intercept   : AIC=inf, Time=3.71 sec
 ARIMA(1,0,0)(0,0,2)[12] intercept   : AIC=-14567.077, 

c:\Users\MSI\Downloads\Datasets\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [29]:
# forecasting for further 30 days to display on dashboard
forecasting_days =30
monaragala_forecast_range = pd.date_range(start=monaragala['Current_date'].max(), periods=forecasting_days+1, freq='D')[1:] # gives the dates of prediction
monaragala_forecast_values = monaragala_arima_fit.forecast(steps=forecasting_days) # gives the predicted HCHO emission values

#getting the predictions onto a data frame
monaragala_forecast_df = pd.DataFrame({'Prediction_date': monaragala_forecast_range, 'HCHO_emission': monaragala_forecast_values, 'Location': 'monaragala'})
monaragala_forecast_df.set_index('Prediction_date', inplace=True)
monaragala_forecast_df

HCHO_emission    Location
Prediction_date                           
2023-12-26            0.000129  monaragala
2023-12-27            0.000134  monaragala
2023-12-28            0.000137  monaragala
2023-12-29            0.000128  monaragala
2023-12-30            0.000128  monaragala
2023-12-31            0.000133  monaragala
2024-01-01            0.000133  monaragala
2024-01-02            0.000134  monaragala
2024-01-03            0.000133  monaragala
2024-01-04            0.000133  monaragala
2024-01-05            0.000134  monaragala
2024-01-06            0.000134  monaragala
2024-01-07            0.000134  monaragala
2024-01-08            0.000134  monaragala
2024-01-09            0.000134  monaragala
2024-01-10            0.000135  monaragala
2024-01-11            0.000135  monaragala
2024-01-12            0.000135  monaragala
2024-01-13            0.000135  monaragala
2024-01-14            0.000135  monaragala
2024-01-15            0.000135  monaragala
2024-01-16            0.000135  monaragala
2024-01-17            0.000135  monaragala
2024-01-18            0.000135  monaragala
2024-01-19            0.000135  monaragala
2024-01-20            0.000135  monaragala
2024-01-21            0.000135  monaragala
2024-01-22            0.000135  monaragala
2024-01-23            0.000135  monaragala
2024-01-24            0.000135  monaragala

In [30]:
folder = 'Forecasted-Predictions'

if not os.path.exists(folder):
    os.makedirs(folder)

monaragala_forecast_df.to_csv(f'{folder}/monaragala_forecast.csv') # save the forecasted data to a csv file

## Jaffna

In [31]:
jaffna = pd.read_csv('Preprocessed-Data/jaffna.csv')
jaffna

HCHO_emission    City Current_date
0          0.000058  Jaffna   2019-01-01
1          0.000102  Jaffna   2019-01-02
2          0.000039  Jaffna   2019-01-03
3          0.000108  Jaffna   2019-01-04
4          0.000159  Jaffna   2019-01-05
...             ...     ...          ...
1415       0.000108  Jaffna   2023-12-26
1416       0.000051  Jaffna   2023-12-28
1417       0.000127  Jaffna   2023-12-29
1418       0.000117  Jaffna   2023-12-30
1419       0.000065  Jaffna   2023-12-31

[1420 rows x 3 columns]

In [32]:
remove_location(jaffna)
date_to_datetime(jaffna)
jaffna


HCHO_emission Current_date
0          0.000058   2019-01-01
1          0.000102   2019-01-02
2          0.000039   2019-01-03
3          0.000108   2019-01-04
4          0.000159   2019-01-05
...             ...          ...
1415       0.000108   2023-12-26
1416       0.000051   2023-12-28
1417       0.000127   2023-12-29
1418       0.000117   2023-12-30
1419       0.000065   2023-12-31

[1420 rows x 2 columns]

In [33]:
split_jaffna = int(len(jaffna) * 0.8) # 80% training split
train_split_jaffna, test_split_jaffna = jaffna.iloc[:split_jaffna], jaffna.iloc[split_jaffna:] # splitting to training and testing

jaffna_auto_arima = auto_arima(train_split_jaffna['HCHO_emission'], seasonal=True, trace=True, m=12) # best parameters are obtained using autoarima
p,d,q = jaffna_auto_arima.order
jaffna_arima = ARIMA(train_split_jaffna['HCHO_emission'], order=(p,d,q)) # creating an ARIMA model
jaffna_arima_fit = jaffna_arima.fit() # fitting the model

jaffna_testing_predictions = jaffna_arima_fit.forecast(steps=len(test_split_jaffna)) # taking predictions on testing data
jaffna_mse = mean_squared_error(test_split_jaffna['HCHO_emission'], jaffna_testing_predictions) # calculating mean squared error
print("Mean Squared Error for jaffna: ", jaffna_mse)
print(jaffna_testing_predictions)

Performing stepwise search to minimize aic


 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=inf, Time=6.71 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=-19022.888, Time=0.07 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=-18518.687, Time=1.42 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=-19027.838, Time=1.34 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=-17271.345, Time=0.07 sec
 ARIMA(0,0,1)(0,0,0)[12] intercept   : AIC=-19027.462, Time=0.58 sec
 ARIMA(0,0,1)(1,0,1)[12] intercept   : AIC=inf, Time=2.76 sec
 ARIMA(0,0,1)(0,0,2)[12] intercept   : AIC=-19027.789, Time=3.68 sec
 ARIMA(0,0,1)(1,0,0)[12] intercept   : AIC=-18515.167, Time=1.67 sec
 ARIMA(0,0,1)(1,0,2)[12] intercept   : AIC=inf, Time=7.52 sec
 ARIMA(0,0,0)(0,0,1)[12] intercept   : AIC=-19023.402, Time=0.87 sec
 ARIMA(1,0,1)(0,0,1)[12] intercept   : AIC=-19039.342, Time=1.84 sec
 ARIMA(1,0,1)(0,0,0)[12] intercept   : AIC=-19039.237, Time=0.67 sec
 ARIMA(1,0,1)(1,0,1)[12] intercept   : AIC=inf, Time=3.64 sec
 ARIMA(1,0,1)(0,0,2)[12] intercept   : AIC=-19038.055, Time=3.

In [34]:
# forecasting for further 30 days to display on dashboard
forecasting_days =30
jaffna_forecast_range = pd.date_range(start=jaffna['Current_date'].max(), periods=forecasting_days+1, freq='D')[1:] # gives the dates of prediction
jaffna_forecast_values = jaffna_arima_fit.forecast(steps=forecasting_days) # gives the predicted HCHO emission values

#getting the predictions onto a data frame
jaffna_forecast_df = pd.DataFrame({'Prediction_date': jaffna_forecast_range, 'HCHO_emission': jaffna_forecast_values, 'Location': 'jaffna'})
jaffna_forecast_df.set_index('Prediction_date', inplace=True)
jaffna_forecast_df

HCHO_emission Location
Prediction_date                        
2024-01-01            0.000097   jaffna
2024-01-02            0.000101   jaffna
2024-01-03            0.000104   jaffna
2024-01-04            0.000104   jaffna
2024-01-05            0.000106   jaffna
2024-01-06            0.000106   jaffna
2024-01-07            0.000106   jaffna
2024-01-08            0.000106   jaffna
2024-01-09            0.000107   jaffna
2024-01-10            0.000107   jaffna
2024-01-11            0.000107   jaffna
2024-01-12            0.000107   jaffna
2024-01-13            0.000107   jaffna
2024-01-14            0.000107   jaffna
2024-01-15            0.000107   jaffna
2024-01-16            0.000107   jaffna
2024-01-17            0.000107   jaffna
2024-01-18            0.000107   jaffna
2024-01-19            0.000107   jaffna
2024-01-20            0.000107   jaffna
2024-01-21            0.000107   jaffna
2024-01-22            0.000107   jaffna
2024-01-23            0.000107   jaffna
2024-01-24            0.000107   jaffna
2024-01-25            0.000107   jaffna
2024-01-26            0.000107   jaffna
2024-01-27            0.000107   jaffna
2024-01-28            0.000107   jaffna
2024-01-29            0.000107   jaffna
2024-01-30            0.000107   jaffna

In [35]:
folder = 'Forecasted-Predictions'

if not os.path.exists(folder):
    os.makedirs(folder)

jaffna_forecast_df.to_csv(f'{folder}/jaffna_forecast.csv') # save the forecasted data to a csv file

## Kurunegala

In [36]:
kurunegala = pd.read_csv('Preprocessed-Data/kurunegala.csv')
kurunegala

HCHO_emission        City Current_date
0          0.000088  Kurunegala   2019-01-01
1          0.000137  Kurunegala   2019-01-02
2          0.000148  Kurunegala   2019-01-03
3          0.000138  Kurunegala   2019-01-04
4          0.000249  Kurunegala   2019-01-05
...             ...         ...          ...
1208       0.000063  Kurunegala   2023-12-23
1209       0.000094  Kurunegala   2023-12-25
1210       0.000152  Kurunegala   2023-12-26
1211       0.000075  Kurunegala   2023-12-27
1212       0.000158  Kurunegala   2023-12-30

[1213 rows x 3 columns]

In [37]:
remove_location(kurunegala)
date_to_datetime(kurunegala)
kurunegala

HCHO_emission Current_date
0          0.000088   2019-01-01
1          0.000137   2019-01-02
2          0.000148   2019-01-03
3          0.000138   2019-01-04
4          0.000249   2019-01-05
...             ...          ...
1208       0.000063   2023-12-23
1209       0.000094   2023-12-25
1210       0.000152   2023-12-26
1211       0.000075   2023-12-27
1212       0.000158   2023-12-30

[1213 rows x 2 columns]

In [38]:
split_kurunegala = int(len(kurunegala) * 0.8) # 80% training split
train_split_kurunegala, test_split_kurunegala = kurunegala.iloc[:split_kurunegala], kurunegala.iloc[split_kurunegala:] # splitting to training and testing

kurunegala_auto_arima = auto_arima(train_split_kurunegala['HCHO_emission'], seasonal=True, trace=True, m=12) # best parameters are obtained using autoarima
p,d,q = kurunegala_auto_arima.order
kurunegala_arima = ARIMA(train_split_kurunegala['HCHO_emission'], order=(p,d,q)) # creating an ARIMA model
kurunegala_arima_fit = kurunegala_arima.fit() # fitting the model
kurunegala_testing_predictions = kurunegala_arima_fit.forecast(steps=len(test_split_kurunegala)) # taking predictions on testing data
kurunegala_mse = mean_squared_error(test_split_kurunegala['HCHO_emission'], kurunegala_testing_predictions) # calculating mean squared error
print("Mean Squared Error for kurunegala: ", kurunegala_mse)
print(kurunegala_testing_predictions)

Performing stepwise search to minimize aic


 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=-15824.894, Time=3.90 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=-15800.941, Time=0.09 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=-15421.632, Time=0.66 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=-15834.069, Time=1.06 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=-14263.711, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[12] intercept   : AIC=-15820.371, Time=0.44 sec
 ARIMA(0,0,1)(1,0,1)[12] intercept   : AIC=-15398.893, Time=2.35 sec
 ARIMA(0,0,1)(0,0,2)[12] intercept   : AIC=-15830.975, Time=3.21 sec
 ARIMA(0,0,1)(1,0,0)[12] intercept   : AIC=-15440.995, Time=1.24 sec
 ARIMA(0,0,1)(1,0,2)[12] intercept   : AIC=inf, Time=6.37 sec
 ARIMA(0,0,0)(0,0,1)[12] intercept   : AIC=-15816.989, Time=1.03 sec
 ARIMA(1,0,1)(0,0,1)[12] intercept   : AIC=-15873.578, Time=1.34 sec
 ARIMA(1,0,1)(0,0,0)[12] intercept   : AIC=-15870.979, Time=0.30 sec
 ARIMA(1,0,1)(1,0,1)[12] intercept   : AIC=-15773.063, Time=2.52 sec
 ARIMA(1,0,1)(0,0,2)[12] intercept   : AI

c:\Users\MSI\Downloads\Datasets\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [39]:
# forecasting for further 30 days to display on dashboard
forecasting_days =30
kurunegala_forecast_range = pd.date_range(start=kurunegala['Current_date'].max(), periods=forecasting_days+1, freq='D')[1:] # gives the dates of prediction
kurunegala_forecast_values = kurunegala_arima_fit.forecast(steps=forecasting_days) # gives the predicted HCHO emission values

#getting the predictions onto a data frame
kurunegala_forecast_df = pd.DataFrame({'Prediction_date': kurunegala_forecast_range, 'HCHO_emission': kurunegala_forecast_values, 'Location': 'kurunegala'})
kurunegala_forecast_df.set_index('Prediction_date', inplace=True)
kurunegala_forecast_df

HCHO_emission    Location
Prediction_date                           
2023-12-31            0.000114  kurunegala
2024-01-01            0.000120  kurunegala
2024-01-02            0.000120  kurunegala
2024-01-03            0.000124  kurunegala
2024-01-04            0.000128  kurunegala
2024-01-05            0.000129  kurunegala
2024-01-06            0.000131  kurunegala
2024-01-07            0.000132  kurunegala
2024-01-08            0.000133  kurunegala
2024-01-09            0.000134  kurunegala
2024-01-10            0.000135  kurunegala
2024-01-11            0.000135  kurunegala
2024-01-12            0.000136  kurunegala
2024-01-13            0.000136  kurunegala
2024-01-14            0.000137  kurunegala
2024-01-15            0.000137  kurunegala
2024-01-16            0.000137  kurunegala
2024-01-17            0.000137  kurunegala
2024-01-18            0.000138  kurunegala
2024-01-19            0.000138  kurunegala
2024-01-20            0.000138  kurunegala
2024-01-21            0.000138  kurunegala
2024-01-22            0.000138  kurunegala
2024-01-23            0.000138  kurunegala
2024-01-24            0.000138  kurunegala
2024-01-25            0.000138  kurunegala
2024-01-26            0.000138  kurunegala
2024-01-27            0.000138  kurunegala
2024-01-28            0.000138  kurunegala
2024-01-29            0.000138  kurunegala

In [40]:
folder = 'Forecasted-Predictions'

if not os.path.exists(folder):
    os.makedirs(folder)

kurunegala_forecast_df.to_csv(f'{folder}/kurunegala_forecast.csv') # save the forecasted data to a csv file